## Lựa chọn đặc trưng đệ quy sử dụng độ quan trọng của Random Forest

Random Forest chỉ định độ quan trọng bằng nhau hoặc tương tự cho các đặc trưng có tính tương quan cao. Ngoài ra, khi các đặc trưng tương quan, độ quan trọng được gán sẽ thấp hơn so với độ quan trọng được gán cho chính đặc trưng đó nếu xây dựng cây mà không có các đặc trưng tương quan.

Do đó, thay vì loại bỏ đặc trưng theo độ quan trọng bằng brute force như chúng ta đã làm trong notebook trước, chúng ta có lựa chọn tốt hơn là loại từng đặc trưng và tính toán lại độ quan trọng ở mỗi lượt. Quy trình này là loại bỏ đặc trưng đệ quy (RFE).

RFE là sự kết hợp giữa phương pháp nhúng và phương pháp gói: dựa trên tính toán thu được khi khớp mô hình, nhưng cũng yêu cầu khớp một số mô hình.

Chu kỳ như sau:

- Xây dựng random forest với tất cả các đặc trưng
- Loại đặc trưng ít quan trọng nhất
- Xây dựng random forest và tính lại độ quan trọng
- Lặp lại cho đến khi thỏa mãn một số tiêu chí

Trong trường hợp này, khi một đặc trưng có tính tương quan cao với đặc trưng khác bị loại thì độ quan trọng của đặc trưng còn lại sẽ tăng. Điều này có thể dẫn đến lựa chọn không gian đặc trưng tốt hơn. Mặt khác, việc xây dựng một số random forest khá tốn thời gian và tài nguyên tính toán, đặc biệt là nếu tập dữ liệu có nhiều đặc trưng.

Chúng ta sẽ minh họa cách lựa chọn đặc trưng đệ quy dựa trên độ quan trọng của random forest, sử dụng sklearn trong tập dữ liệu phân loai.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE
from sklearn.metrics import roc_auc_score

In [2]:
# load tập dữ liệu
data = pd.read_csv('dataset_2.csv')
data.shape

(50000, 109)

In [3]:
data.head()

,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,var_10,...,var_100,var_101,var_102,var_103,var_104,var_105,var_106,var_107,var_108,var_109
0,4.532710,3.280834,17.982476,4.404259,2.349910,0.603264,2.784655,0.323146,12.009691,0.139346,...,2.079066,6.748819,2.941445,18.360496,17.726613,7.774031,1.473441,1.973832,0.976806,2.541417
1,5.821374,12.098722,13.309151,4.125599,1.045386,1.832035,1.833494,0.709090,8.652883,0.102757,...,2.479789,7.795290,3.557890,17.383378,15.193423,8.263673,1.878108,0.567939,1.018818,1.416433
2,1.938776,7.952752,0.972671,3.459267,1.935782,0.621463,2.338139,0.344948,9.937850,11.691283,...,1.861487,6.130886,3.401064,15.850471,14.620599,6.849776,1.098210,1.959183,1.575493,1.857893
3,6.020690,9.900544,17.869637,4.366715,1.973693,2.026012,2.853025,0.674847,11.816859,0.011151,...,1.340944,7.240058,2.417235,15.194609,13.553772,7.229971,0.835158,2.234482,0.946170,2.700606
4,3.909506,10.576516,0.934191,3.419572,1.871438,3.340811,1.868282,0.439865,13.585620,1.153366,...,2.738095,6.565509,4.341414,15.893832,11.929787,6.954033,1.853364,0.511027,2.599562,0.811364


**Quan trọng**

Trong tất cả các quy trình lựa chọn đặc trưng, chỉ nên chọn các đặc trưng bằng cách kiểm tra tập huấn luyện, điều này giúp tránh overfit.

In [4]:
# tách thành tập huấn luyện và tập kiểm tra
## Yêu cầu 1:
## VIẾT CODE Ở ĐÂY:
X_train, X_test, y_train, y_test = train_test_split(
    data.drop(labels=['target'], axis=1),
    data['target'],
    test_size=0.3,
    random_state=0)

X_train.shape, X_test.shape

((35000, 108), (15000, 108))

<details><summary> Gợi ý </summary>

[train_test_split()](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)

</details>

### Lựa chọn đặc trưng cách đệ quy

In [5]:
# chúng ta thực hiện huấn luyện mô hình và lựa chọn đặc trưng trong 2 dòng code

# đầu tiên, chỉ định Random Forest và các tham số

# sau đó là RFE từ sklearn để loại bỏ đặc trưng cách đệ quy

# RFE sẽ loại một đặc trưng ở mỗi lần lặp => ít quan trọng nhất
# sau đó nó xây một random forest khác và lặp lại
# cho tới khi đáp ứng một số tiêu chí.

# trong sklearn, tiêu chí dừng là số lượng 
# đặc trưng tùy ý được chọn mà chúng ta cần quyết định trước
# không cần giải pháp tốt nhất, có giải pháp là được
## Yêu cầu 2:
## VIẾT CODE Ở ĐÂY:
sel_ = RFE(RandomForestClassifier(n_estimators=10, random_state=10), n_features_to_select=27)
sel_.fit(X_train, y_train)

RFE(estimator=RandomForestClassifier(n_estimators=10, random_state=10),
    n_features_to_select=27)

<details><summary> Gợi ý </summary>

[RFE()](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html)

</details>

In [6]:
# các đặc trưng được chọn
## Yêu cầu 3:
## VIẾT CODE Ở ĐÂY:
selected_feat = X_train.columns[(sel_.get_support()]
len(selected_feat)

SyntaxError: closing parenthesis ']' does not match opening parenthesis '(' (363580541.py, line 4)

<details><summary> Gợi ý </summary>

Sử dụng ```get_support()```

</details>

In [ ]:
# hãy hiển thị danh sách các đặc trưng
selected_feat

Index(['var_4', 'var_14', 'var_21', 'var_25', 'var_30', 'var_31', 'var_34',
       'var_38', 'var_41', 'var_46', 'var_53', 'var_55', 'var_56', 'var_70',
       'var_71', 'var_73', 'var_79', 'var_82', 'var_86', 'var_87', 'var_88',
       'var_90', 'var_92', 'var_93', 'var_96', 'var_104', 'var_108'],
      dtype='object')

In [ ]:
# có các đặc trưng đã chọn trong
# notebook trước, trong đó chúng ta sử dụng SelectFromModel từ sklearn
# mà không cần thực hiện theo cách đệ quy

previous_lecture_selected_features = [
    'var_1', 'var_2', 'var_6', 'var_9', 'var_13', 'var_15', 'var_16', 'var_17',
    'var_20', 'var_21', 'var_30', 'var_34', 'var_37', 'var_55', 'var_60',
    'var_67', 'var_69', 'var_70', 'var_71', 'var_82', 'var_87', 'var_88',
    'var_95', 'var_96', 'var_99', 'var_103', 'var_108'
]

### So sánh chất lượng của các tập con đặc trưng

In [ ]:
# tạo một hàm để xây random forest và so sánh
# chất lượng của chúng trong tập huấn luyện và tập kiểm tra

def run_randomForests(X_train, X_test, y_train, y_test):
    
    rf = RandomForestClassifier(n_estimators=200, random_state=39, max_depth=4)
    rf.fit(X_train, y_train)
    ## Yêu cầu 4:
    ## VIẾT CODE Ở ĐÂY:
    print('Train set')
    pred = rf.predict_proba(X_train)
    print('Random Forests roc-auc: {}'.format(roc_auc_score(y_train, pred[:,1])))
    
    print('Test set')
    pred = rf.predict_proba(X_test)
    print('Random Forests roc-auc: {}'.format(roc_auc_score(y_test, pred[:,1])))

<details><summary> Gợi ý </summary>

Sử dụng ```predict_proba()``` và ```roc_auc_score()```

</details>

In [ ]:
# các đặc trưng được lựa chọn theo cách đệ quy
## Yêu cầu 5:
## VIẾT CODE Ở ĐÂY:
run_randomForests(X_train[selected_feat],
                  X_test[selected_feat],
                  y_train, y_test)

<details><summary> Gợi ý </summary>

Sử dụng ```run_randomForests()```

</details>

In [ ]:
# các đặc trưng được lựa chọn toàn bộ
## Yêu cầu 6:
## VIẾT CODE Ở ĐÂY:
run_randomForests(X_train[previous_lecture_selected_features], X_test[previous_lecture_selected_features], y_train, y_test)

<details><summary> Gợi ý </summary>

Sử dụng ```run_randomForests()``` và ```previous_lecture_selected_features```

</details>

Chúng ta thấy rằng RFE không trả về tập hợp con tốt hơn của các đặc trưng. Chất lượng của mô hình đã xây bằng các biến được lựa chọn trực tiếp từ Random Forest đầu tiên đủ để tìm ra tập hợp con tốt của các đặc trưng.

Theo tôi, RFE từ sklearn không đem lại lợi thế lớn so với phương thức SelectFromModel, và tôi thường dùng SelectFromModel để lựa chọn đặc trưng.
